In [223]:
import torch
from torch import nn
import torchvision
from torch.utils import data
from torchvision import transforms
from d2l import torch as d2l
from torch.nn import functional as F
from torch.optim import lr_scheduler
import math
def get_dataloader_workers():
    return 4
def load_data_CIFAR100(batch_size,train_augs=None,test_augs=None,resize=None):  
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    if train_augs == None:
        CIFAR100_train = torchvision.datasets.CIFAR100(root="/root/autodl-tmp/data",train=True,transform=trans,download=True)
        CIFAR100_test = torchvision.datasets.CIFAR100(root="/root/autodl-tmp/data",train=False,transform=trans,download=True)
    else:
        CIFAR100_train = torchvision.datasets.CIFAR100(root="/root/autodl-tmp/data",train=True,transform=train_augs,download=True)
        CIFAR100_test = torchvision.datasets.CIFAR100(root="/root/autodl-tmp/data",train=False,transform=test_augs,download=True)
    return (data.DataLoader(CIFAR100_train, batch_size, shuffle=True,num_workers=get_dataloader_workers()),
            data.DataLoader(CIFAR100_test, batch_size, shuffle=False,num_workers=get_dataloader_workers()))

In [ ]:
finetune_net = torchvision.models.resnet18(pretrained=True)
finetune_net.fc = nn.Linear(finetune_net.fc.in_features, 100)
nn.init.kaiming_uniform_(finetune_net.fc.weight)

In [229]:
def train(net, train_iter, test_iter, num_epochs, loss, trainer, device,scheduler=None):
    # def init_weights(m):
    #     if type(m) == nn.Linear or type(m) == nn.Conv2d:
    #         nn.init.xavier_uniform_(m.weight)
    # net.apply(init_weights)
    # print('training on', device)
    net.to(device)
    loss = loss
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],legend=['train loss', 'train acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)
    for epoch in range(num_epochs):
        metric = d2l.Accumulator(4)
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            X,y=features,labels
            X = X.to(device)
            y = y.to(device)
            net.train()
            trainer.zero_grad()
            pred = net(X)
            l = loss(pred, y)
            l.sum().backward()
            trainer.step()
            train_loss_sum = l.sum()
            train_acc_sum = d2l.accuracy(pred, y)
            metric.add(train_loss_sum, train_acc_sum, labels.shape[0], labels.numel())
            timer.stop()
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (metric[0] / metric[2], metric[1] / metric[3],
                              None))
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
        if scheduler:
            if scheduler.__module__==lr_scheduler.__name__:
                scheduler.step()
            else:
                for param_group in trainer.param_groups:
                    param_group['lr']=scheduler(epoch)
                    
            
    print(f'loss {metric[0] / metric[2]:.3f}, train acc '
          f'{metric[1] / metric[3]:.3f}, test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec on '
          f'{str(device)}')

In [230]:
def train_fine_tuning(net, train_iter, test_iter,lr, num_epochs, loss, device,
                      scheduler=None,
                      param_group=True):
    if param_group:
        params_1x = [
            param for name, param in net.named_parameters()
            if name not in ["fc.weight", "fc.bias"]]
        trainer = torch.optim.Adam([{'params': params_1x},
                                   {'params': net.fc.parameters(),
                                    'lr': lr * 10}],
                                  lr=lr,weight_decay=0.001)
    else:
        trainer = torch.optim.Adam(net.parameters(), lr=lr,weight_decay=0.001)
    train(net, train_iter, test_iter, num_epochs, loss, trainer, device)

In [ ]:
lr,num_epochs, batch_size =5e-5, 8, 128
mean = [0.5070751592371323, 0.48654887331495095, 0.4409178433670343]
std = [0.2673342858792401, 0.2564384629170883, 0.27615047132568404]
train_augs =transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ColorJitter(brightness=0, contrast=0, saturation=0,hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
test_augs=transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
loss=nn.CrossEntropyLoss(reduction="none",label_smoothing=0.1)
device=d2l.try_gpu()
net=finetune_net
train_iter, test_iter = load_data_CIFAR100(batch_size,train_augs=train_augs,test_augs=test_augs)
train_fine_tuning(net, train_iter, test_iter,lr, num_epochs,loss, device)